In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv("oil_production.csv")

In [ ]:
df.drop("Unnamed: 0", axis = 1, inplace = True)

In [ ]:
t = df["DATEPRD"]

In [ ]:
df.drop("DATEPRD",axis = 1, inplace = True)

In [ ]:
X = df.iloc[:,0:6]
Y = df.iloc[:,6]

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split



In [ ]:
x_train = X
y_train = Y

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler_x = MinMaxScaler()
x_train = scaler_x.fit_transform(x_train)#.as_matrix())
#x_val = scaler_x.transform(x_val.as_matrix())
#x_test = scaler_x.transform(x_test.as_matrix())

scaler_y = MinMaxScaler()
y_train = scaler_y.fit_transform(y_train.reshape(-1,1))
#y_val = scaler_y.transform(y_val.reshape(-1,1))
#y_test = scaler_y.transform(y_test.reshape(-1,1))

In [ ]:
X = np.array(x_train)
y = np.array(y_train)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
import plotly.offline as py
py.init_notebook_mode(connected = True)
import plotly.graph_objs as go

In [ ]:
model_rf = RandomForestRegressor(random_state=42, criterion = 'mse',
 max_depth= 8,
 max_features='sqrt',
 n_estimators= 200)
tscvs = TimeSeriesSplit(n_splits=5)

In [ ]:
def train(X):
    x_axis = []
    y_axis = []
    for i, (train_idx, val_idx) in enumerate(tscvs.split(X)):
        print(" \nFold", i)
        X_train_cv = X[train_idx]
        y_train_cv = y[train_idx]
        X_val_cv = X[val_idx]
        y_val_cv = y[val_idx]
        model_rf.fit(X_train_cv, y_train_cv)
        print(model_rf.score(X_val_cv, y_val_cv))
        fig, ax = plt.subplots()
        plt.plot(model_rf.predict(X_val_cv))
        plt.plot(y_val_cv, 'r')
        a, b = val_idx,model_rf.predict(X_val_cv)
        x_axis.append(a)
        y_axis.append(scaler_y.inverse_transform(b.reshape(-1,1)))
    return x_axis , y_axis

In [ ]:
a,b = train(X)

In [ ]:
trace0 = go.Scatter(x = a[0], y = b[0].reshape(-1,), name = "Prediction 1")
trace1 = go.Scatter(x = a[1], y = b[1].reshape(-1,), name = "Prediction 2")
trace2 = go.Scatter(x = a[2], y = b[2].reshape(-1,), name = "Prediction 3")
trace3 = go.Scatter(x = a[3], y = b[3].reshape(-1,), name = "Prediction 4")
trace4 = go.Scatter(x = a[4], y = b[4].reshape(-1,), name = "Prediction 5")
trace5 = go.Scatter(x = np.arange(len(y_train)), y = scaler_y.inverse_transform(y_train).reshape(-1,), name = 'Actual water rate')

layout = go.Layout(
    xaxis=dict(
        #title='Actual oil rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        #title='Predicted oil rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
data =  [trace5,trace0,trace1,trace2,trace3,trace4]
fig = go.Figure(data=data, layout = layout)
py.iplot(fig)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler, Normalizer, MaxAbsScaler, StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score



In [ ]:
x_train.shape[1]

In [ ]:
model = Sequential()
model.add(Dense(30, input_dim=x_train.shape[1], activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(10, activation='relu'))

model.add(Dense(1, activation='linear'))

model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [ ]:
tscvs = TimeSeriesSplit(n_splits=5)
for i, (train_idx, val_idx) in enumerate(tscvs.split(X)):
    print(" \nFold", i)
    X_train_cv = X[train_idx]
    y_train_cv = y[train_idx]
    X_val_cv = X[val_idx]
    y_val_cv = y[val_idx]
    model.fit(X_train_cv, y_train_cv, batch_size = 64, epochs = 200, verbose = 0)
    print(r2_score(y_val_cv, model.predict(X_val_cv)))

In [ ]:
from sklearn.svm import SVR
model_svr =SVR()
def train(X):
    x_axis = []
    y_axis = []
    for i, (train_idx, val_idx) in enumerate(tscvs.split(X)):
        print(" \nFold", i)
        X_train_cv = X[train_idx]
        y_train_cv = y[train_idx]
        X_val_cv = X[val_idx]
        y_val_cv = y[val_idx]
        model_svr.fit(X_train_cv, y_train_cv)
        print(model_svr.score(X_val_cv, y_val_cv))
        fig, ax = plt.subplots()
        plt.plot(model_svr.predict(X_val_cv))
        plt.plot(y_val_cv, 'r')
        a, b = val_idx,model_svr.predict(X_val_cv)
        x_axis.append(a)
        y_axis.append(scaler_y.inverse_transform(b.reshape(-1,1)))
    return x_axis , y_axis

In [ ]:
a, b = train(X)

In [ ]:
from sklearn.svm import SVR
model = SVR(C=10.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma=0.1, kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001)

In [ ]:
def train(model):
    x_axis = []
    y_axis = []
    for i, (train_idx, val_idx) in enumerate(tscvs.split(X)):
        print(" \nFold", i)
        X_train_cv = X[train_idx]
        y_train_cv = y[train_idx]
        X_val_cv = X[val_idx]
        y_val_cv = y[val_idx]
        model.fit(X_train_cv, y_train_cv)
        print(model.score(X_val_cv, y_val_cv))
        #fig, ax = plt.subplots()
        #plt.plot(model.predict(X_val_cv))
        #plt.plot(y_val_cv, 'r')
        a, b = val_idx,model.predict(X_val_cv)
        x_axis.append(a)
        y_axis.append(scaler_y.inverse_transform(b.reshape(-1,1)))
    return x_axis , y_axis

In [ ]:
a,b = train(model)

In [ ]:
trace0 = go.Scatter(x = a[0], y = b[0].reshape(-1,), name = "Prediction 1")
trace1 = go.Scatter(x = a[1], y = b[1].reshape(-1,), name = "Prediction 2")
trace2 = go.Scatter(x = a[2], y = b[2].reshape(-1,), name = "Prediction 3")
trace3 = go.Scatter(x = a[3], y = b[3].reshape(-1,), name = "Prediction 4")
trace4 = go.Scatter(x = a[4], y = b[4].reshape(-1,), name = "Prediction 5")
trace5 = go.Scatter(x = np.arange(len(y_train)), y = scaler_y.inverse_transform(y_train).reshape(-1,),
                    name = 'Actual oil rate')

layout = go.Layout(
    xaxis=dict(
        #title='Actual oil rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        #title='Predicted oil rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
data =  [trace5,trace0,trace1,trace2,trace3,trace4]
fig = go.Figure(data=data, layout = layout)
py.iplot(fig)

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
model_ett = ExtraTreesRegressor(n_estimators=200, n_jobs=4, min_samples_split=25,
                            min_samples_leaf=35, max_features='auto')
model_ett.fit(x_train, y_train)

In [ ]:
a,b = train(model_ett)

In [ ]:
trace0 = go.Scatter(x = a[0], y = b[0].reshape(-1,), name = "Prediction 1")
trace1 = go.Scatter(x = a[1], y = b[1].reshape(-1,), name = "Prediction 2")
trace2 = go.Scatter(x = a[2], y = b[2].reshape(-1,), name = "Prediction 3")
trace3 = go.Scatter(x = a[3], y = b[3].reshape(-1,), name = "Prediction 4")
trace4 = go.Scatter(x = a[4], y = b[4].reshape(-1,), name = "Prediction 5")
trace5 = go.Scatter(x = np.arange(len(y_train)), y = scaler_y.inverse_transform(y_train).reshape(-1,),
                    name = 'Actual oil rate')

layout = go.Layout(
    xaxis=dict(
        #title='Actual oil rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        #title='Predicted oil rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
data =  [trace5,trace0,trace1,trace2,trace3,trace4]
fig = go.Figure(data=data, layout = layout)
py.iplot(fig)

In [ ]:
import xgboost as xgb
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [ ]:
a,b = train(model_xgb)

In [ ]:
trace0 = go.Scatter(x = a[0], y = b[0].reshape(-1,), name = "Prediction 1")
trace1 = go.Scatter(x = a[1], y = b[1].reshape(-1,), name = "Prediction 2")
trace2 = go.Scatter(x = a[2], y = b[2].reshape(-1,), name = "Prediction 3")
trace3 = go.Scatter(x = a[3], y = b[3].reshape(-1,), name = "Prediction 4")
trace4 = go.Scatter(x = a[4], y = b[4].reshape(-1,), name = "Prediction 5")
trace5 = go.Scatter(x = np.arange(len(y_train)), y = scaler_y.inverse_transform(y_train).reshape(-1,),
                    name = 'Actual oil rate')

layout = go.Layout(
    xaxis=dict(
        #title='Actual oil rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        #title='Predicted oil rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
data =  [trace5,trace0,trace1,trace2,trace3,trace4]
fig = go.Figure(data=data, layout = layout)
py.iplot(fig)

In [ ]:
import lightgbm as lgb
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=64,
                              learning_rate=0.005, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [ ]:
a,b = train(model_lgb)

In [ ]:
trace0 = go.Scatter(x = a[0], y = b[0].reshape(-1,), name = "Prediction 1")
trace1 = go.Scatter(x = a[1], y = b[1].reshape(-1,), name = "Prediction 2")
trace2 = go.Scatter(x = a[2], y = b[2].reshape(-1,), name = "Prediction 3")
trace3 = go.Scatter(x = a[3], y = b[3].reshape(-1,), name = "Prediction 4")
trace4 = go.Scatter(x = a[4], y = b[4].reshape(-1,), name = "Prediction 5")
trace5 = go.Scatter(x = np.arange(len(y_train)), y = scaler_y.inverse_transform(y_train).reshape(-1,),
                    name = 'Actual oil rate')

layout = go.Layout(
    xaxis=dict(
        #title='Actual oil rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        #title='Predicted oil rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
data =  [trace5,trace0,trace1,trace2,trace3,trace4]
fig = go.Figure(data=data, layout = layout)
py.iplot(fig)